In [5]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install spotipy

In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [2]:
# Spotify API 인증 설정
client_id = "9bd86ce2eea34cccb8961d47792e9796"  # Spotify Developer Dashboard에서 발급받은 Client ID
client_secret = "71c37f5b363a47ca8ce1248ed4f3dfaa"  # Spotify Developer Dashboard에서 발급받은 Client Secret

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [4]:
def get_tracks_by_keyword(keyword, limit=10):
    """
    Spotify API를 통해 특정 키워드로 트랙 검색
    :param keyword: 검색 키워드 (e.g., "relaxing music")
    :param limit: 가져올 곡의 수
    :return: 곡 정보 리스트
    """
    results = sp.search(q=keyword, limit=limit, type="track") # 키워드 검색 해 트랙 가져오기
    tracks = [] # 트랙별 정보 딕셔너리가 담긴 리스트(name, )
    for item in results['tracks']['items']:
        # 트랙별 정보
        track_info = {
            "BGM_name": item['name'],  # 곡 이름
            "BGM_url": item['external_urls']['spotify'],  # Spotify URL
            "artist": ", ".join([artist['name'] for artist in item['artists']]),  # 아티스트 이름들 (여러 명일 경우 ','로 연결)
            "album_cover": item['album']['images'][0]['url'] if item['album']['images'] else None  # 앨범 커버 이미지 URL (이미지가 없으면 None)
        }
        tracks.append(track_info)  # 리스트에 추가
    return tracks

In [7]:
import pymysql

def save_to_database(tracks):
    """
    곡 정보를 MySQL 데이터베이스에 저장
    :param tracks: 곡 정보 리스트
    """
    connection = pymysql.connect(
        host="localhost",
        port= 3306,
        user = 'root',
        password="1111",
        database="prome_15",
        charset="utf8mb4"
    ) # sql 커넥션
    try:
        with connection.cursor() as cursor:
            for track in tracks:
                sql = """
                INSERT INTO bgm_track (BGM_name, BGM_url,artist,album_cover)
                VALUES (%s, %s, %s, %s)
                """
                cursor.execute(sql, (track['BGM_name'], track['BGM_url'],track['artist'], track['album_cover']))
        connection.commit()
    finally:
        connection.close()
        


In [8]:

# 데이터 저장 예제
if __name__ == "__main__":
    tracks = get_tracks_by_keyword("아이유유", limit=5)
    save_to_database(tracks)
